# Multiclass classification using a neural network

Here we adopt a 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, train_test_split, KFold

import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13501108782085776789
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3137627750
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4215924836951942508
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
matrix = pd.read_csv('model/1129-fixed-data-matrix-karlgren.csv').set_index('character')

In [3]:
import json
with open('hypothesized_phonetic_series.json', encoding='utf8') as f:
    js = json.load(f)

In [4]:
def getps(char):
    if char in js:
        return js[char]
    else:
        return 0
    
def rowIndex(row):
    return getps(row.name)

matrix['ps'] = matrix.apply(rowIndex, axis=1)

one_hot = pd.get_dummies(matrix['ps'], prefix = 'ps')
dropval = 3
print(one_hot.shape)
one_hot.drop([col for col, val in one_hot.sum().iteritems() if val < dropval], axis=1, inplace=True)
print(one_hot.shape)

matrix = matrix.drop('ps', axis = 1)

matrix = matrix.join(one_hot)
matrix.columns
matrix.to_csv("out.csv")

(15250, 4097)
(15250, 981)


In [5]:
cols = matrix.columns
features_cols = [c for c in cols if 'Karlgren' not in c and 'tone_label' not in c]
labels_cols = [c for c in cols if c not in features_cols]

In [6]:
X_complete = matrix[features_cols]
Y_complete = matrix[labels_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X_complete, Y_complete, test_size=.3, random_state=0)

In [21]:
label_categories = [
    ('tone_label', 
     (
        (32, 'relu'),
        (16, 'relu'),
     ),
     10
    ),
    ('Karlgren_onset',
     (
        (768, 'relu'),
        (0.8, 'dropout'),
        (192, 'relu'),
     ),
     20
    ),
    ('Karlgren_nucleus',
     (
        (1024, 'relu'),
        (0.8, 'dropout'),
        (192, 'relu'),
     ),
     20
    ),
    ('Karlgren_coda',
     (
        (96, 'relu'),
        (24, 'relu'),
     ),
     10
    )
]

In [22]:
def fit_keras_models(X, X_test, Y, Y_test):
    category_to_performance = defaultdict(list)
    category_to_test_performance = defaultdict(list)
    category_to_callbacks = defaultdict(list)
    
    for category, architecture, num_epochs in label_categories:
        print("======= Starting training for " + category + " =======")

        Y_subset = Y[[x for x in Y.columns if category in x]]
        Y_test_subset = Y_test[[x for x in Y_test.columns if category in x]]
        n_bins = Y_subset.shape[1]
        
        def build_keras_nn_model():
            model = Sequential()
            model.add(Dense(
                architecture[0][0],
                input_dim=X.shape[1],
                activation=architecture[0][1])
            )
            
            if len(architecture) > 1:
                for l, (hidden_layer_size, act_fn) in enumerate(architecture[1:]):
                    if act_fn is 'dropout':
                        model.add(Dropout(
                            rate=hidden_layer_size
                        ))
                    else:
                        model.add(Dense(
                            hidden_layer_size,
                            input_dim=architecture[l - 1],
                            activation=act_fn)
                        )
                    
            model.add(Dense(n_bins, activation='softmax'))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            return model
        
        KerasNN = KerasClassifier(
            build_fn=build_keras_nn_model,
            epochs=num_epochs,
            batch_size=256,
            verbose=True
        )
        
        X_train = X
        Y_train = Y_subset

        hist_callback = KerasNN.fit(
            X_train,
            Y_train,
        )

        category_to_callbacks[category].append(hist_callback)
        test_acc = KerasNN.score(X_test, Y_test_subset)
        category_to_test_performance[category].append(test_acc)
        print('Test performance: {:.3f}'.format(float(test_acc)))
        print('\n\n')
    return category_to_performance, category_to_callbacks, category_to_test_performance

In [33]:
performance, callbacks, test_performance = fit_keras_models(
    X_train,
    X_test,
    Y_train,
    Y_test,
)
print("DONE")

======= Starting training for tone_label =======
Epoch 1/10
10698/10698 [==============================] - 2s 226us/step - loss: 1.2534 - acc: 0.4459
Epoch 2/10
10698/10698 [==============================] - 1s 49us/step - loss: 0.8903 - acc: 0.6983
Epoch 3/10
10698/10698 [==============================] - 1s 51us/step - loss: 0.6344 - acc: 0.7934
Epoch 4/10
10698/10698 [==============================] - 1s 49us/step - loss: 0.5298 - acc: 0.8090
Epoch 5/10
10698/10698 [==============================] - 1s 50us/step - loss: 0.4893 - acc: 0.8192
Epoch 6/10
10698/10698 [==============================] - 1s 50us/step - loss: 0.4650 - acc: 0.8312
Epoch 7/10
10698/10698 [==============================] - 1s 60us/step - loss: 0.4472 - acc: 0.8375: 0s - loss: 0.4504 - acc: 0
Epoch 8/10
10698/10698 [==============================] - 1s 63us/step - loss: 0.4323 - acc: 0.8440
Epoch 9/10
10698/10698 [==============================] - 1s 62us/step - loss: 0.4192 - acc: 0.8474: 0s - loss: 0.4106 - a

In [34]:
test_performance

defaultdict(list,
            {'tone_label': [0.8113824690578808],
             'Karlgren_onset': [0.7634103787657583],
             'Karlgren_nucleus': [0.7503270832305905],
             'Karlgren_coda': [0.9387265578191768]})